# 2024년 12월 25일 금요일
## 할일
- [x] 오전: 딥러닝 강의시청
- [x] 오후: 수학 특강

## ANN (Artificial Neural Network) : 정형 데이터 추론
가장 기본적인 형태의 신경망으로, 여러 개의 퍼셉트론(뉴런)을 층(layer)로 쌓아서 구성된다.
입력층, 

In [ ]:
import torch
import torch.nn as nn # neural network
import torch.optim as optim # 최적화 부분
import torchvision # 이미지 처리
import torchvision.transforms as transforms # 전처리 라이브러리

In [ ]:
# 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Normalize 0.5 0.5는 

# MNIST 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
# 다운로드 경로, train set인지 여부, 다운로드 할건지, transform 할건지(전처리 한 상태로 받을건지)
# batch_size 64개로 조깸, shuffle = 섞어서 배치를 만들어라

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [ ]:
trainloader

In [ ]:
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 입력층에서 은닉층으로
        self.fc2 = nn.Linear(128, 64)       # 은닉층에서 은닉층으로
        self.fc3 = nn.Linear(64, 10)        # 은닉층에서 출력층으로

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 입력 이미지를 1차원 벡터로 변환
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

클래스를 정해주고, 상속받아야 함
두가지 해줘야함. 
1) __init__ 함수 만들기
fc (fully connected layer) = 1 차원
nn.Linear는 입력과 출력이 몇개씩인지 알려줘야함
28*28은 MNIST의 데이터 크기임
2) forward 함수 지정하기
    x.view에서 -1은 2차원에서 마음대로 정해라 하는 뜻
   torch.relu는 활성화 함수 (relu 적용)

### 모델 학습

In [ ]:
# 모델 초기화
model = SimpleANN()

#결과 출력 (학습 하기 전 한번 결과를 출력하는 코드를 넣어봤음) 
correct = 0
total = 0
with torch.no_grad(): #no_grad는 메모리 효율 부분때문에 적어둔 것. 평가단계에서 기울기를 계산하지 않음. 
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
 #SGD는 가중치를 업데이트 할때 쓰는 방법 중 하나. 

# 모델 학습
for epoch in range(10):  # 10 에포크 동안 학습
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward() # 기울기 찾기
        optimizer.step() #가중치 ...

        # 손실 출력
        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

Accuracy of the network on the 10000 test images: 11.59%
[Epoch 1, Batch 100] loss: 1.325
[Epoch 1, Batch 200] loss: 0.474
[Epoch 1, Batch 300] loss: 0.423
[Epoch 1, Batch 400] loss: 0.354
[Epoch 1, Batch 500] loss: 0.311
[Epoch 1, Batch 600] loss: 0.278
[Epoch 1, Batch 700] loss: 0.249
[Epoch 1, Batch 800] loss: 0.258
[Epoch 1, Batch 900] loss: 0.203
[Epoch 2, Batch 100] loss: 0.214
[Epoch 2, Batch 200] loss: 0.200
[Epoch 2, Batch 300] loss: 0.184
[Epoch 2, Batch 400] loss: 0.190
[Epoch 2, Batch 500] loss: 0.184
[Epoch 2, Batch 600] loss: 0.171
[Epoch 2, Batch 700] loss: 0.144
[Epoch 2, Batch 800] loss: 0.155
[Epoch 2, Batch 900] loss: 0.156
[Epoch 3, Batch 100] loss: 0.134
[Epoch 3, Batch 200] loss: 0.129
[Epoch 3, Batch 300] loss: 0.137
[Epoch 3, Batch 400] loss: 0.141
[Epoch 3, Batch 500] loss: 0.138
[Epoch 3, Batch 600] loss: 0.133
[Epoch 3, Batch 700] loss: 0.124
[Epoch 3, Batch 800] loss: 0.111
[Epoch 3, Batch 900] loss: 0.126
[Epoch 4, Batch 100] loss: 0.109
[Epoch 4, Batch 200

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(images, labels)

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., 

나머지 CNN, RNN,LSTM 작성 부분은 DL 폴더에 있음